# عوامل هوش مصنوعی Azure با پشتیبانی از پروتکل زمینه مدل (MCP)

این دفترچه نشان می‌دهد که چگونه می‌توان از عوامل هوش مصنوعی Azure با ابزارهای پروتکل زمینه مدل (MCP) استفاده کرد تا یک عامل هوشمند ایجاد کرد که بتواند از سرورهای خارجی MCP برای قابلیت‌های پیشرفته بهره‌مند شود.


## نصب بسته‌های مورد نیاز NuGet

ابتدا باید بسته Azure AI Agents Persistent را نصب کنیم که قابلیت‌های اصلی برای کار با Azure AI Agents را فراهم می‌کند.


## مزایای احراز هویت بدون کلید

این دفترچه **احراز هویت بدون کلید** را نشان می‌دهد که چندین مزیت ارائه می‌دهد:
- ✅ **بدون نیاز به مدیریت کلیدهای API** - از احراز هویت مبتنی بر هویت Azure استفاده می‌کند
- ✅ **امنیت بیشتر** - هیچ راز یا اطلاعات حساسی در کد یا تنظیمات ذخیره نمی‌شود
- ✅ **چرخش خودکار اعتبارنامه‌ها** - Azure چرخه عمر اعتبارنامه‌ها را مدیریت می‌کند
- ✅ **دسترسی مبتنی بر نقش** - از RBAC Azure برای مجوزهای دقیق استفاده می‌کند

`DefaultAzureCredential` به طور خودکار بهترین منبع اعتبارنامه موجود را استفاده می‌کند:
1. هویت مدیریت‌شده (هنگام اجرا در Azure)
2. اعتبارنامه‌های Azure CLI (در طول توسعه)
3. اعتبارنامه‌های Visual Studio
4. متغیرهای محیطی (در صورت پیکربندی)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

بسته Azure Identity را برای احراز هویت با خدمات Azure با استفاده از DefaultAzureCredential نصب کنید.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## وارد کردن فضای نام‌های مورد نیاز

فضای نام‌های ضروری برای Azure AI Agents و Azure Identity را وارد کنید.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## تنظیم کلاینت Azure AI Agent (احراز هویت بدون کلید)

متغیرهای پیکربندی را تنظیم کنید و PersistentAgentsClient را با استفاده از **احراز هویت بدون کلید** ایجاد کنید:
- **projectEndpoint**: نقطه پایانی پروژه Azure AI Foundry
- **modelDeploymentName**: نام مدل هوش مصنوعی مستقر شده (GPT-4.1 nano)
- **mcpServerUrl**: آدرس URL سرور MCP (Microsoft Learn API)
- **mcpServerLabel**: یک برچسب برای شناسایی سرور MCP
- **DefaultAzureCredential**: از هویت مدیریت‌شده، Azure CLI یا منابع دیگر اعتبارنامه استفاده می‌کند (نیازی به کلیدهای API نیست)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## ایجاد تعریف ابزار MCP

تعریف ابزاری برای MCP ایجاد کنید که به سرور MCP مایکروسافت لرن متصل شود. این کار به عامل اجازه می‌دهد به محتوای مایکروسافت لرن و مستندات دسترسی پیدا کند.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## ایجاد عامل هوش مصنوعی

یک عامل هوش مصنوعی پایدار با مدل مشخص شده و ابزارهای MCP ایجاد کنید. این عامل با موارد زیر پیکربندی شده است:
- مدل GPT-4.1 nano  
- دستورالعمل‌هایی برای استفاده از ابزارهای MCP جهت کمک  
- دسترسی به سرور MCP مایکروسافت لرن  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## ایجاد رشته گفتگو و ارسال پیام

یک رشته گفتگو ایجاد کنید و یک پیام کاربر ارسال کنید که درباره تفاوت بین Azure OpenAI و OpenAI سؤال می‌کند. این کار توانایی عامل را در استفاده از ابزارهای MCP برای ارائه اطلاعات دقیق آزمایش می‌کند.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## تنظیم منابع ابزار MCP (بدون کلید)

منابع ابزار MCP را تنظیم کنید. برای یک روش کاملاً بدون کلید، می‌توانید هدرهای سفارشی را حذف کنید اگر سرور MCP از احراز هویت مبتنی بر هویت Azure پشتیبانی می‌کند. مثال زیر نشان می‌دهد که چگونه هدرها را اضافه کنید در صورت نیاز، اما برای سناریوهای بدون کلید، ممکن است این‌ها ضروری نباشند.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## شروع اجرای عامل

ایجاد و شروع یک اجرا برای پردازش پیام کاربر. عامل از ابزارها و منابع تنظیم‌شده MCP برای تولید پاسخ استفاده خواهد کرد.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## نظارت بر اجرای عامل و مدیریت تأیید ابزارها (بدون کلید)

وضعیت اجرای عامل را نظارت کنید و هرگونه تأیید ابزار مورد نیاز را مدیریت کنید. این حلقه:
1. منتظر می‌ماند تا اجرا کامل شود یا نیاز به اقدام داشته باشد
2. به‌طور خودکار درخواست‌های ابزار MCP را در صورت نیاز تأیید می‌کند
3. برای احراز هویت بدون کلید، ممکن است نیازی به هدرها نباشد اگر سرور MCP از هویت Azure پشتیبانی کند


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## نمایش نتایج مکالمه

تمام پیام‌های موجود در رشته مکالمه را بازیابی و نمایش دهید، به‌طوری‌که هم سؤال کاربر و هم پاسخ نماینده نشان داده شوند. پیام‌ها به ترتیب زمانی همراه با نشانگرهای زمان و نقش نمایش داده می‌شوند.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
